<a href="https://colab.research.google.com/github/Adith-blusim/BP-/blob/main/common_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Deleting all values of "sys" > 140 and 'dias' values > 100:
#seconds data

import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/blusim/roi_updated_data.csv")
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop(['Unix Timestamp','RESP','ECG','SPO2','PR'],axis=1)

def extract_nibp_values(value):
    # Remove spaces and extra brackets, extract the digits
    cleaned_value = ''.join(filter(str.isdigit, value.replace(' ', '').replace('(', '').replace(')', '')))
    sys_value = cleaned_value[:3]
    dias_value = cleaned_value[3:5]
    return sys_value, dias_value

# Apply the function to "NIBP" column and create new columns "sys" and "dias"
df['sys'], df['dias'] = zip(*df['NIBP'].apply(extract_nibp_values))

rdf = pd.read_csv("/content/drive/MyDrive/blusim/mat_15.csv")
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%H:%M:%S')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M:%S')

df.set_index('Timestamp', inplace=True)
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()
rdf.set_index('Timestamp', inplace=True)
df=df.resample('S').ffill()
df.index = df.index.strftime('%H:%M:%S')

ldf = pd.merge(rdf,df, left_index=True, right_index=True)
ldf[['sys', 'dias']] = ldf[['sys', 'dias']].replace('', np.nan)
ldf.dropna(how='any', inplace=True)
ldf['sys'] = pd.to_numeric(ldf['sys'], errors='coerce')
ldf['dias'] = pd.to_numeric(ldf['dias'], errors='coerce')
ldf = ldf.drop(ldf[ldf['sys'].gt(140)].index)
ldf = ldf.drop(ldf[ldf['dias'].gt(100)].index)
ldf

In [ ]:
#Interpolation before signal processing

from scipy.interpolate import PchipInterpolator

def interpolate_to_120(arr):

    pchip_interpolator = PchipInterpolator(np.arange(len(arr)), arr)
    x_interp = np.linspace(0, len(arr) - 1, 120)
    y_interp = pchip_interpolator(x_interp)
    return y_interp.tolist()

df_interpolated = ldf[['Raw_Value_ch1','Raw_Value_ch2','Raw_Value_ch3','Raw_Value_ch4']].applymap(interpolate_to_120)
df_interpolated

In [ ]:
!pip install EMD-signal
from PyEMD import EMD
from PyEMD import EEMD
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, hilbert
# Sample rate and desired cutoff frequencies (in Hz)
fs = 138.0
lowcut = 0.5
highcut = 6.0
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
# Apply the Butterworth band-pass filter to the data
bdf = pd.DataFrame()
bdf['buttered_channel1'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Raw_Value_ch1']]
bdf['buttered_channel2'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Raw_Value_ch2']]
bdf['buttered_channel3'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Raw_Value_ch3']]
bdf['buttered_channel4'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Raw_Value_ch4']]

eemd = EMD()
imf1c1 = []
imf1c2 = []
imf1c3 = []
imf1c4 = []

for i in bdf['buttered_channel1']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c1.append(i1)

for i in bdf['buttered_channel2']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c2.append(i1)

for i in bdf['buttered_channel3']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c3.append(i1)

for i in bdf['buttered_channel4']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c4.append(i1)

ins_phase_c1 = []
ins_phase_c2 = []
ins_phase_c3 = []
ins_phase_c4 = []

for i in imf1c1:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c1.append(instantaneous_phase)

for i in imf1c2:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c2.append(instantaneous_phase)

for i in imf1c3:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c3.append(instantaneous_phase)

for i in imf1c4:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c4.append(instantaneous_phase)



In [ ]:
#Interpolation after signal processing

from scipy.interpolate import PchipInterpolator

def interpolate_to_120(arr):

    pchip_interpolator = PchipInterpolator(np.arange(len(arr)), arr)
    x_interp = np.linspace(0, len(arr) - 1, 120)
    y_interp = pchip_interpolator(x_interp)
    return y_interp.tolist()

ipc1 = [interpolate_to_120(lst) for lst in ins_phase_c1]
ipc2 = [interpolate_to_120(lst) for lst in ins_phase_c2]
ipc3 = [interpolate_to_120(lst) for lst in ins_phase_c3]
ipc4 = [interpolate_to_120(lst) for lst in ins_phase_c4]


print(max(len(x) for x in ipc1))

In [ ]:
# standarizing array-size in each cell to be the same as the size of the smallest array
minlen = (min(len(x) for x in ins_phase_c1))
ipc1 = [lst[:minlen] for lst in ins_phase_c1 if len(lst) >= minlen]
ipc2 = [lst[:minlen] for lst in ins_phase_c2 if len(lst) >= minlen]
ipc3 = [lst[:minlen] for lst in ins_phase_c3 if len(lst) >= minlen]
ipc4 = [lst[:minlen] for lst in ins_phase_c4 if len(lst) >= minlen]

print(max(len(x) for x in ipc1))

In [ ]:
from sklearn.model_selection import train_test_split
merged_ipc = np.stack((ipc1,ipc2,ipc3,ipc4), axis=1)
merged_ipc = np.swapaxes(merged_ipc, 1, 2)
print(merged_ipc.shape)

X_train = merged_ipc
Y_train = ldf[['sys', 'dias']].to_numpy().astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization
tf.keras.backend.clear_session()

input_shape =  (120, 4)
model = Sequential()

#first Conv layer
model.add(Conv1D(filters=100, kernel_size=21, strides=1, input_shape=input_shape))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#second Conv layer
model.add(Conv1D(filters=200, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#third Conv layer
model.add(Conv1D(filters=300, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())
#Final dense layer
model.add(Dense(2, activation='linear'))

model.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.summary()
history = model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), batch_size=64)

In [ ]:
predictions = model.predict(X_test)
Y_pred = []
yf = pd.DataFrame(predictions, columns=['predicted_output_1', 'predicted_output_2'])
yf[['true_output_1', 'true_output_2']] = pd.DataFrame(Y_test, columns=['true_output_1', 'true_output_2'])
# yf = pd.DataFrame(Y_pred, columns=['predicted'])
yf


In [ ]:
absolute_percentage_error = np.abs((yf[['true_output_1', 'true_output_2']].values - yf[['predicted_output_1', 'predicted_output_2']].values) / yf[['true_output_1', 'true_output_2']].values)

# Calculate accuracy for each row
accuracy_per_row = 1 - absolute_percentage_error

# Set your accuracy threshold 90%
accuracy_threshold = 0.9

# Filter rows where both predictions have accuracy >= accuracy_threshold
accurate_rows = yf[(accuracy_per_row >= accuracy_threshold).all(axis=1)]

# Print the DataFrame with only accurate rows
print("Accurate Rows:")
print(accurate_rows)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Plotting training and validation loss
plt.figure(figsize=(10, 6))

# Training loss
plt.plot(history.history['loss'], label='Training Loss', marker='o')

# Validation loss
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Performance')
plt.legend()

# Adding grid for better readability
plt.grid(True)

# Show plot
plt.show()

In [ ]:
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['figure.facecolor'] = 'black'

sample_indices = np.arange(len(yf))
plt.figure(figsize=(20,5))
# Plotting line graph for output 1
plt.plot(sample_indices, yf['true_output_1'], label='Systolic(Actual)', linestyle='-', color='blue')
plt.plot(sample_indices, yf['predicted_output_1'], label='Systolic(Predicted)', linestyle='--', color='cyan')

# Plotting line graph for output 2
plt.plot(sample_indices, yf['true_output_2'], label='Diastolic(Actual)', linestyle='-', color='green')
plt.plot(sample_indices, yf['predicted_output_2'], label='Diastolic(Predicted)', linestyle='--', color='lime')

# Set labels and title
plt.xlabel('Sample Index',color='white')
plt.ylabel('Output Values',color='white')
plt.title('Comparison of Predicted and True Outputs',color='white')
legend = plt.legend()
for text in legend.get_texts():
    text.set_color("white")

# Show the plot
plt.show()